# Gene-Taxa Downstream Analysis

## Download Data

In [22]:
!wget -P gene-taxa-dataset https://zenodo.org/api/records/12964684/files-archive


--2024-10-17 15:21:35--  https://zenodo.org/api/records/12964684/files-archive
Resolving zenodo.org (zenodo.org)... 188.184.103.159, 188.185.79.172, 188.184.98.238, ...
Connecting to zenodo.org (zenodo.org)|188.184.103.159|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘gene-taxa-dataset/files-archive’

files-archive           [    <=>             ] 945.97M  18.2MB/s    in 73s     

2024-10-17 15:22:49 (13.0 MB/s) - ‘gene-taxa-dataset/files-archive’ saved [991921356]



In [24]:
!unzip gene-taxa-dataset/files-archive -d gene-taxa-dataset
!rm gene-taxa-dataset/files-archive


Archive:  gene-taxa-dataset/files-archive
 extracting: gene-taxa-dataset/val.fasta  
 extracting: gene-taxa-dataset/hierarchical-level.txt  
 extracting: gene-taxa-dataset/test.fasta  
 extracting: gene-taxa-dataset/gene_out.fasta  
 extracting: gene-taxa-dataset/taxa_out.fasta  
 extracting: gene-taxa-dataset/train.fasta  
 extracting: gene-taxa-dataset/metadata.csv  


## Kraken2

#### Rename the Header of FASTA File for Compatibility with Kraken


In [26]:
import os
from Bio import SeqIO

# Directory containing the FASTA files
fasta_dir = "gene-taxa-dataset"

# Loop through all files in the directory
for file_name in os.listdir(fasta_dir):
    if file_name.endswith(".fasta"):  # Check if it's a FASTA file
        input_file = os.path.join(fasta_dir, file_name)
        output_file = os.path.join(fasta_dir, f"{file_name.split('.')[0]}_kraken_like.fasta")
        
        # Open the output file
        with open(output_file, "w") as outfile:
            # Process each sequence in the FASTA file
            with open(input_file, "r") as infile:
                for record in SeqIO.parse(infile, "fasta"):
                    # Split the header and rearrange the parts
                    parts = record.id.split('|')
                    new_header = f">{parts[3]}|{parts[0]}|{parts[1]}|kraken:taxid|{parts[3]}"
                    
                    # Write the modified header and sequence to the output file
                    outfile.write(f"{new_header}\n{record.seq}\n")


##### It is required to download the latest version of Kraken2. The version we used is from 23 November 2023.


### Download Taxanomy db 

In [13]:
!singularity exec --fakeroot --bind {os.getcwd()}:/data kraken2_latest.sif kraken2-build --download-taxonomy --db /data/data-gene/databases

INFO:    User not listed in /etc/subuid, trying root-mapped namespace
INFO:    Using fakeroot command combined with root-mapped namespace
INFO:    underlay of /etc/localtime required more than 50 (79) bind mounts

### Create db from train.fasta file 

In [27]:
import os

!singularity exec --fakeroot --bind {os.getcwd()}:/data kraken2_latest.sif kraken2-build --add-to-library /data/gene-taxa-dataset/train_kraken_like.fasta --db /data/data-gene/databases


INFO:    User not listed in /etc/subuid, trying root-mapped namespace
INFO:    Using fakeroot command combined with root-mapped namespace
INFO:    underlay of /etc/localtime required more than 50 (79) bind mounts
Masking low-complexity regions of new file... done.
Added "/data/gene-taxa-dataset/train_kraken_like.fasta" to library (/data/data-gene/databases)


### Infrence on taxa_out for Kraken

In [28]:
! mkdir taxa_out_output
!singularity exec --fakeroot --bind {os.getcwd()}:/data kraken2_latest.sif \
kraken2 --db /data/data-gene/databases \
--output /data/taxa_out_output/kraken2_results.txt \
--report /data/taxa_out_output/kraken2_report.txt \
--minimum-hit-groups 1 \
--confidence 0 \
/data/gene-taxa-dataset/taxa_out_kraken_like.fasta

mkdir: cannot create directory ‘taxa_out_output’: File exists
INFO:    User not listed in /etc/subuid, trying root-mapped namespace
INFO:    Using fakeroot command combined with root-mapped namespace
INFO:    underlay of /etc/localtime required more than 50 (79) bind mounts
Loading database information... done.
11800 sequences (12.38 Mbp) processed in 1.202s (589.0 Kseq/m, 617.98 Mbp/m).
  5896 sequences classified (49.97%)
  5904 sequences unclassified (50.03%)


In [29]:
import pandas as pd
def traverse_nodes(initial_taxid, nodes_dict):
    taxids = [initial_taxid]
    ranks = []
    wanted_ranks = np.array(['superkingdom','kingdom', 'phylum', 'class', 'order', 'family', 'genus', 'species'])
    
    while initial_taxid in nodes_dict.keys():
        taxid, rank = nodes_dict[initial_taxid]
        taxids.append(taxid)
        ranks.append(rank)
        initial_taxid = taxid
        if initial_taxid == 1:
            break
    ranks = np.array(ranks)
    mask = np.nonzero(np.in1d(ranks, wanted_ranks))[0]
        
    taxids = list(np.array(taxids)[mask])
    ranks = list(np.array(ranks)[mask])
    zipped_taxids = dict(zip(ranks, taxids))

    return zipped_taxids

#tree file
nodes = pd.read_csv('./data-gene/databases/taxonomy/nodes.dmp', delimiter = "\t", header=None, skipinitialspace=True, usecols=[0,2,4])
nodes.rename(columns={0: 'taxid', 2: 'prev_taxid', 4: 'rank'}, inplace=True)

#taxid -> name file
names = pd.read_csv('./data-gene/databases/taxonomy/names.dmp', delimiter = "\t", header=None, skipinitialspace=True, usecols=[0,2])
names.rename(columns={0: 'taxid', 2: 'name'}, inplace=True)



In [33]:
import numpy as np
#List of taxids
basic = pd.read_csv('./taxa_out_output/kraken2_results.txt', delimiter = "\t", header=None, skipinitialspace=True) # usecols=[0,2])

basic["orginal_index"] = basic[1].str.extract(r'seqid\|(\d+)', expand=False).astype(int)
basic["taxid"] = basic[1].str.extract(r'(\d+)|', expand=False).astype(int)

basic.rename(columns={2: 'pred_taxid'}, inplace=True)

#traverse tree
values = list(zip(nodes['prev_taxid'], nodes['rank']))
keys = list(nodes['taxid'])
nodes_dict = dict(map(lambda i,j : (i,j) , keys,values))
 

In [34]:

full_taxa = basic.copy()
full_taxa['real_taxa'] = full_taxa.apply(lambda row: traverse_nodes(row['taxid'], nodes_dict), axis=1)
                                                  
lineage = pd.DataFrame(list(full_taxa['real_taxa'])).fillna(0).astype('Int64') # lineage from taxid  

full_taxa = basic.copy()
full_taxa['real_taxa'] = full_taxa.apply(lambda row: traverse_nodes(row['pred_taxid'], nodes_dict), axis=1)
lineage_pred = pd.DataFrame(list(full_taxa['real_taxa'])).fillna(0).astype('Int64') # lineage from taxid   

In [6]:
import pandas as pd


# Initialize accuracy dictionary to store accuracy for each level
accuracy = {}

# Loop through each level
for level in lineage.columns:
    # Check if the level exists in both DataFrames
    if level in lineage_pred.columns:
        # Count the number of matching values at this level, while ignoring rows where lineage_pred value is zero
        matches = ((lineage[level] == lineage_pred[level]) & (lineage_pred[level] != 0)).sum()
        # Count the total number of non-zero values in lineage_pred at this level
        total_values = len(lineage)
        # Calculate accuracy
        if total_values != 0:
            level_accuracy = matches / total_values
        else:
            level_accuracy = None
        # Store accuracy for this level
        accuracy[level] = level_accuracy

# Print accuracy for each level
for level, acc in accuracy.items():
    print(f"Accuracy for level {level}: {acc}")



## Mmseq2

In [36]:
!mmseqs easy-search gene-taxa-dataset/taxa_out.fasta gene-taxa-dataset/train.fasta ./taxa_out.m8 tmp --max-accept 1  --max-seqs 1 --search-type 3


Create directory tmp
easy-search gene-taxa-dataset/taxa_out.fasta gene-taxa-dataset/train.fasta ./taxa_out.m8 tmp --max-accept 1 --max-seqs 1 --search-type 3 

MMseqs Version:                        	f6c98807d589091c625db68da258d587795acbab
Substitution matrix                    	aa:blosum62.out,nucl:nucleotide.out
Add backtrace                          	false
Alignment mode                         	3
Alignment mode                         	0
Allow wrapped scoring                  	false
E-value threshold                      	0.001
Seq. id. threshold                     	0
Min alignment length                   	0
Seq. id. mode                          	0
Alternative alignments                 	0
Coverage threshold                     	0
Coverage mode                          	0
Max sequence length                    	65535
Compositional bias                     	1
Compositional bias                     	1
Max reject                             	2147483647
Max accept                  

## BERTax

### Download BERTax

We have **Singularity** available, but you could alternatively use the Docker file provided by DeepMicrobes to set up the environment.


In [ ]:
!singularity pull --tmpdir $(pwd) $(pwd)/bertax.sif docker://fkre/bertax

### Run Bertax

In [ ]:
!singularity run --bind ${PWD}:/mnt  --nv bertax.sif  -o /mnt/gene-taxa-dataset/result_test.txt   /mnt/gene-taxa-dataset/test.fasta  --batch_size 128


## DeepMicrobes

#### Running DeepMicrobes: Guidelines

Please follow the detailed DeepMicrobes documentation for running their model and formatting processes. We explicitly follow their documentation for the two key steps:

1. **Create TFRecord**  
   Follow the instructions in the DeepMicrobes documentation to generate TFRecord files:  
   [Create TFRecord Documentation](https://github.com/MicrobeLab/DeepMicrobes/blob/master/document/tfrecord.md)

2. **Train DeepMicrobes**  
   Refer to the DeepMicrobes training documentation to train the model:  
   [Train DeepMicrobes Documentation](https://github.com/MicrobeLab/DeepMicrobes/blob/master/document/train.md)

---
###### Adaptation in Our Work
We also included the two core parameters from DeepMicrobes to train our own DeepMicrobes-based model on the gene-taxa dataset.  
We trained two different DeepMicrobes models: one for family and the other for gene, to ensure a fair comparison on the same dataset.



In [ ]:
!tfrec_train_kmer.sh -i train_gene_label.fasta -v ./DeepMicrobes/tokens_merged_8mers.txt -o train_gene.tfrec -k 8 -s 2000

In [ ]:
!python3 -u DeepMicrobes.py \
  --input_tfrec train_gene.tfrec \
  --model_name attention \
  --model_dir gene_model \
  --num_classes 437 \
  --vocab_size 32898 \
  --train_epochs 20 \
  --batch_size 256 \
  --max_len 2048

# ART Simulation Analysis

### Install Simulator

We suggest running the following command in the terminal instead of in a notebook!


In [2]:
!wget https://www.niehs.nih.gov/sites/default/files/2024-02/artsrcmountrainier2016.06.05linux.tgz
!tar -xvzf artsrcmountrainier2016.06.05linux.tgz

--2024-11-09 18:57:22--  https://www.niehs.nih.gov/sites/default/files/2024-02/artsrcmountrainier2016.06.05linux.tgz
Resolving www.niehs.nih.gov (www.niehs.nih.gov)... 104.18.14.200, 104.18.15.200, 2606:4700::6812:fc8, ...
Connecting to www.niehs.nih.gov (www.niehs.nih.gov)|104.18.14.200|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12242989 (12M) [application/x-gzip]
Saving to: ‘artsrcmountrainier2016.06.05linux.tgz’

artsrcmountrainier2 100%[===================>]  11.68M  --.-KB/s    in 0.1s    

2024-11-09 18:57:22 (86.7 MB/s) - ‘artsrcmountrainier2016.06.05linux.tgz’ saved [12242989/12242989]



In [4]:
cd art_src_MountRainier_Linux

In [5]:
!./configure 
!make
!make install

### Generate ART Simulated Data

In [ ]:
!./art_src_MountRainier_Linux/art_illumina -ss HS25 -i ./gene-taxa-dataset/gene_out.fasta -l 150 -f 20 -m 200 -s 10 -o simulated_art

!awk 'NR % 4 == 1 {print ">" substr($0, 2)} NR % 4 == 2 {print}' simulated_art.fq> gene_out_art.fasta

###  Kraken2 Inference

##### We just used the database created in the previous section of Kraken

In [1]:
!mkdir -p ART_output

In [5]:
import os

In [25]:
!singularity exec --fakeroot --bind {os.getcwd()}:/data kraken2_latest.sif \
    kraken2 --db /data/data-gene/databases \
    --output /data/ART_output/kraken2_results.txt \
    --report /data/ART_output/kraken2_report.txt \
    --minimum-hit-groups 1 \
    --confidence 0 \
    /data/gene_out_art.fasta


INFO:    User not listed in /etc/subuid, trying root-mapped namespace
INFO:    Using fakeroot command combined with root-mapped namespace
INFO:    underlay of /etc/localtime required more than 50 (79) bind mounts
Loading database information... done.
2969056 sequences (445.36 Mbp) processed in 20.621s (8638.8 Kseq/m, 1295.82 Mbp/m).
  11258 sequences classified (0.38%)
  2957798 sequences unclassified (99.62%)


In [26]:
import pandas as pd
def traverse_nodes(initial_taxid, nodes_dict):
    taxids = [initial_taxid]
    ranks = []
    wanted_ranks = np.array(['superkingdom','kingdom', 'phylum', 'class', 'order', 'family', 'genus', 'species'])
    
    while initial_taxid in nodes_dict.keys():
        taxid, rank = nodes_dict[initial_taxid]
        taxids.append(taxid)
        ranks.append(rank)
        initial_taxid = taxid
        if initial_taxid == 1:
            break
    ranks = np.array(ranks)
    mask = np.nonzero(np.in1d(ranks, wanted_ranks))[0]
        
    taxids = list(np.array(taxids)[mask])
    ranks = list(np.array(ranks)[mask])
    zipped_taxids = dict(zip(ranks, taxids))

    return zipped_taxids

#tree file
nodes = pd.read_csv('./data-gene/databases/taxonomy/nodes.dmp', delimiter = "\t", header=None, skipinitialspace=True, usecols=[0,2,4])
nodes.rename(columns={0: 'taxid', 2: 'prev_taxid', 4: 'rank'}, inplace=True)

#taxid -> name file
names = pd.read_csv('./data-gene/databases/taxonomy/names.dmp', delimiter = "\t", header=None, skipinitialspace=True, usecols=[0,2])
names.rename(columns={0: 'taxid', 2: 'name'}, inplace=True)



In [27]:

import numpy as np
#List of taxids
basic = pd.read_csv('./ART_output/kraken2_results.txt', delimiter = "\t", header=None, skipinitialspace=True) # usecols=[0,2])

basic["orginal_index"] = basic[1].str.extract(r'seqid\|(\d+)', expand=False).astype(int)
basic["taxid"] = basic[1].str.extract(r'taxid\|(\d+)', expand=False).astype(int)

basic.rename(columns={2: 'pred_taxid'}, inplace=True)

#traverse tree
values = list(zip(nodes['prev_taxid'], nodes['rank']))
keys = list(nodes['taxid'])
nodes_dict = dict(map(lambda i,j : (i,j) , keys,values))
 

In [28]:

full_taxa = basic.copy()
full_taxa['real_taxa'] = full_taxa.apply(lambda row: traverse_nodes(row['taxid'], nodes_dict), axis=1)
                                                  
lineage = pd.DataFrame(list(full_taxa['real_taxa'])).fillna(0).astype('Int64') # lineage from taxid  

full_taxa = basic.copy()
full_taxa['real_taxa'] = full_taxa.apply(lambda row: traverse_nodes(row['pred_taxid'], nodes_dict), axis=1)
lineage_pred = pd.DataFrame(list(full_taxa['real_taxa'])).fillna(0).astype('Int64') # lineage from taxid   

In [2]:
import pandas as pd


# Initialize accuracy dictionary to store accuracy for each level
accuracy = {}

# Loop through each level
for level in lineage.columns:
    # Check if the level exists in both DataFrames
    if level in lineage_pred.columns:
        # Count the number of matching values at this level, while ignoring rows where lineage_pred value is zero
        matches = ((lineage[level] == lineage_pred[level]) & (lineage_pred[level] != 0)).sum()
        # Count the total number of non-zero values in lineage_pred at this level
        total_values = len(lineage)
        # Calculate accuracy
        if total_values != 0:
            level_accuracy = matches / total_values
        else:
            level_accuracy = None
        # Store accuracy for this level
        accuracy[level] = level_accuracy

# Print accuracy for each level
for level, acc in accuracy.items():
    print(f"Accuracy for level {level}: {acc * 100}") 

 

In [3]:
from sklearn.metrics import confusion_matrix, f1_score

for level in lineage.columns:
    # Filter out zero predictions
    ground_truth = lineage[level][lineage_pred[level] != 0]
    predictions = lineage_pred[level][lineage_pred[level] != 0]
    
    # Calculate the confusion matrix
    cm = confusion_matrix(ground_truth, predictions)

    
    # Calculate F1-score with macro averaging
    level_f1_score = f1_score(ground_truth, predictions, average='macro', zero_division=0)
    print(f"F1-Score for {level}: {level_f1_score * 100}")
